In [ ]:
import os,csv,re, time
import pickle
import random
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from scipy import stats
from scipy.sparse import issparse
import scanpy as sc
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import cv2
import ST as ST
# from IPython.display import Image

counts=sc.read("/home/dingsq/dsq/MGCL-ST-project/Data/CSCC/sample_data.h5ad")
#Read in hitology image

img=cv2.imread("/home/dingsq/dsq/MGCL-ST-project/Data/CSCC/76ab255be098f0210db462c6e2ec9c5.jpg")

resize_factor=1000/np.min(img.shape[0:2])
resize_width=int(img.shape[1]*resize_factor)
resize_height=int(img.shape[0]*resize_factor)
counts.var.index=[i.upper() for i in counts.var.index]
counts.var_names_make_unique()
counts.raw=counts
sc.pp.log1p(counts) # impute on log scale
if issparse(counts.X):counts.X=counts.X.A

In [ ]:
#Three different algorithms to detect contour, select the best one.Here we use cv2.

#-----------------1. Detect contour using cv2-----------------
cnt=ST.cv2_detect_contour(img, apertureSize=5,L2gradient = True)

binary=np.zeros((img.shape[0:2]), dtype=np.uint8)
cv2.drawContours(binary, [cnt], -1, (1), thickness=-1)
#Enlarged filter
cnt_enlarged = ST.scale_contour(cnt, 1.05)
binary_enlarged = np.zeros(img.shape[0:2])
cv2.drawContours(binary_enlarged, [cnt_enlarged], -1, (1), thickness=-1)
img_new = img.copy()
cv2.drawContours(img_new, [cnt], -1, (255), thickness=50)
img_new=cv2.resize(img_new, ((resize_width, resize_height)))
cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/cnt.jpg', img_new)


In [ ]:
res=50
# Note, if the numer of superpixels is too large and take too long, you can increase the res to 100
enhanced_exp_adata=ST.imputation(img=img, raw=counts, cnt=cnt, genes=counts.var.index.tolist(), shape="None", res=res, s=1, k=2, num_nbs=10)

In [ ]:
enhanced_exp_adata.write_h5ad("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/sample_adata_imputation.h5ad")

In [ ]:
enhanced_exp_adata=sc.read_h5ad("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/sample_adata_imputation.h5ad")

In [ ]:
genes=['BUB1B', 'KIF1C','TOP2A', 'CD151', 'MMP10', 'PTHLH','FEZ1','IL24','KCNMA','INHBA','MAGEA4','NT5E','LAMC2','SLITRK6']
genes=list(set([i for i in genes if i in enhanced_exp_adata.var.index ]))
#target_size can be set to "small" or "large".
pred_refined, target_clusters, c_m=ST.annotation(img=img, 
                                                    binary=binary,
                                                    sudo_adata=enhanced_exp_adata, 
                                                    genes=genes, 
                                                    resize_factor=resize_factor,
                                                    target_size="small")
#Plot
ret_img=ST.visualize_annotation(img=img, 
                              binary=binary, 
                              resize_factor=resize_factor,
                              pred_refined=pred_refined, 
                              target_clusters=target_clusters, 
                              c_m=c_m)

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/tumor.jpg', ret_img)

#Save
np.save("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/tumor_annotation.npy", pred_refined)
print("Target_clusters: ", target_clusters, "\n")
#Save the cluster density information
c_d={i[0]:i[1] for i in c_m[0:len(target_clusters)]}
print("Cluster_density : ", c_d)
with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/tumor_annotation_c_d.pkl', 'wb') as f: pickle.dump(c_d, f)

In [ ]:
#enhanced_exp_adata=sc.read("/home/dingsq/dsq/TESLA_project/results/enhanced_exp_adata.h5ad")
pred_refined=np.load("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/tumor_annotation.npy")
target_clusters=[1, 14, 5, 12, 8, 15, 11, 10, 16, 9, 13]
c_m = [(1, 0.6584800072173188), (14, 0.6077735666772176), (5, 0.5845881322501804), (12, 0.5693501767266952), (8, 0.5059291479350649), (15, 0.502178324743813), (11, 0.47439923506860937), (10, 0.44173677108808934), (16, 0.41465521973410924), (9, 0.39811588367403644), (13, 0.33970737984619676)]

ret_img=ST.leading_edge_detection(img=img, 
                                     pred_refined=pred_refined, 
                                     resize_factor=resize_factor, 
                                     target_clusters=target_clusters, 
                                     binary=binary)

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/leading_edge.jpg', ret_img)

shrink_rate=0.8
res=50
enhanced_exp_adata.obs.index = enhanced_exp_adata.obs.index.astype(str)
binary_tumor, binary_core, core_edge_exp=ST.tumor_edge_core_separation(img=img, 
                                                                           binary=binary,
                                                                           resize_factor=resize_factor, 
                                                                           pred_refined=pred_refined, 
                                                                           target_clusters=target_clusters, 
                                                                           sudo_adata=enhanced_exp_adata, 
                                                                           res=res, 
                                                                           shrink_rate=shrink_rate)

ret_img=ST.plot_tumor_edge_core(img=img, 
                                   resize_factor=resize_factor, 
                                   binary=binary, 
                                   binary_tumor=binary_tumor, 
                                   binary_core=binary_core, 
                                   color_edge=[66, 50, 225], 
                                   color_core=[62, 25, 53])

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/core_edge.jpg', ret_img)

In [ ]:
genes_B=['CD19', 'CD79A', 'CD79B','MS4A1','CD22']
genes_B=list(set([i for i in genes_B if i in enhanced_exp_adata.var.index ]))
#target_size can be set to "small" or "large".
pred_refined_B, target_clusters_B, c_m_B=ST.annotation(img=img, 
                                                    binary=binary,
                                                    sudo_adata=enhanced_exp_adata, 
                                                    genes=genes_B, 
                                                    resize_factor=resize_factor,
                                                    target_size="small")
#Plot
ret_img_B=ST.visualize_annotation(img=img, 
                              binary=binary, 
                              resize_factor=resize_factor,
                              pred_refined=pred_refined_B, 
                              target_clusters=target_clusters_B, 
                              c_m=c_m_B)

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/B.jpg', ret_img_B)

#Save
np.save("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/B_annotation.npy", pred_refined_B)
print("Target_clusters: ", target_clusters_B, "\n")
#Save the cluster density information
c_d_B={i[0]:i[1] for i in c_m_B[0:len(target_clusters_B)]}
print("Cluster_density : ", c_d_B)
with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/B_annotation_c_d.pkl', 'wb') as f: pickle.dump(c_d_B, f)

genes_CD=['CD8A','CD8B']
genes_CD=list(set([i for i in genes_CD if i in enhanced_exp_adata.var.index ]))
#target_size can be set to "small" or "large".
pred_refined_CD, target_clusters_CD, c_m_CD=ST.annotation(img=img, 
                                                    binary=binary,
                                                    sudo_adata=enhanced_exp_adata, 
                                                    genes=genes_CD, 
                                                    resize_factor=resize_factor,
                                                    target_size="small")
#Plot
ret_img_CD=ST.visualize_annotation(img=img, 
                              binary=binary, 
                              resize_factor=resize_factor,
                              pred_refined=pred_refined_CD, 
                              target_clusters=target_clusters_CD, 
                              c_m=c_m_CD)

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CD.jpg', ret_img_CD)

#Save
np.save("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CD_annotation.npy", pred_refined_CD)
print("Target_clusters: ", target_clusters_CD, "\n")
#Save the cluster density information
c_d_CD={i[0]:i[1] for i in c_m_CD[0:len(target_clusters_CD)]}
print("Cluster_density : ", c_d_CD)
with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CD_annotation_c_d.pkl', 'wb') as f: pickle.dump(c_d_CD, f)

genes_DC=['CDIA,', 'CD1B','CDIE','CLEC10A','CLIC2','WFDC21P']
genes_DC=list(set([i for i in genes_DC if i in enhanced_exp_adata.var.index ]))
#target_size can be set to "small" or "large".
pred_refined_DC, target_clusters_DC, c_m_DC=ST.annotation(img=img, 
                                                    binary=binary,
                                                    sudo_adata=enhanced_exp_adata, 
                                                    genes=genes_DC, 
                                                    resize_factor=resize_factor,
                                                    target_size="small")
#Plot
ret_img_DC=ST.visualize_annotation(img=img, 
                              binary=binary, 
                              resize_factor=resize_factor,
                              pred_refined=pred_refined_DC, 
                              target_clusters=target_clusters_DC, 
                              c_m=c_m_DC)

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/DC.jpg', ret_img_DC)

#Save
np.save("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/DC_annotation.npy", pred_refined_DC)
print("Target_clusters: ", target_clusters_DC, "\n")
#Save the cluster density information
c_d_DC={i[0]:i[1] for i in c_m_DC[0:len(target_clusters_DC)]}
print("Cluster_density : ", c_d_DC)
with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/DC_annotation_c_d.pkl', 'wb') as f: pickle.dump(c_d_DC, f)

genes_CXC=['CXCL13']
genes_CXC=list(set([i for i in genes_CXC if i in enhanced_exp_adata.var.index ]))
#target_size can be set to "small" or "large".
pred_refined_CXC, target_clusters_CXC, c_m_CXC=ST.annotation(img=img, 
                                                    binary=binary,
                                                    sudo_adata=enhanced_exp_adata, 
                                                    genes=genes_CXC, 
                                                    resize_factor=resize_factor,
                                                    target_size="small")
#Plot
ret_img_CXC=ST.visualize_annotation(img=img, 
                              binary=binary, 
                              resize_factor=resize_factor,
                              pred_refined=pred_refined_CXC, 
                              target_clusters=target_clusters_CXC, 
                              c_m=c_m_CXC)

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CXC.jpg', ret_img_CXC)

#Save
np.save("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CXC_annotation.npy", pred_refined_CXC)
print("Target_clusters: ", target_clusters_CXC, "\n")
#Save the cluster density information
c_d_CXC={i[0]:i[1] for i in c_m_CXC[0:len(target_clusters_CXC)]}
print("Cluster_density : ", c_d_CXC)
with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CXC_annotation_c_d.pkl', 'wb') as f: pickle.dump(c_d_CXC, f)

In [ ]:
cnt_color = clr.LinearSegmentedColormap.from_list('magma', ["#000003",  "#3b0f6f",  "#8c2980",   "#f66e5b", "#fd9f6c", "#fbfcbf"], N=256)
genes=["IGFBP2", "CXCL12", "CCNB1","IL2RB"]
plot_dir="/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/"
ST.plot_edge_core_enrichd_genes(img=img, 
                                   resize_factor=resize_factor,
                                   binary=binary, 
                                   binary_tumor=binary_tumor, 
                                   sudo_core_edge=core_edge_exp,
                                   genes=genes, 
                                   cnt_color=cnt_color, 
                                   plot_dir=plot_dir, 
                                   res=res)

In [ ]:
cnt_color = clr.LinearSegmentedColormap.from_list('red', ["#EAE7CC", '#BA0000'], N=256)
pred_refined1=np.load("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/B_annotation.npy")
pred_refined2=np.load("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CD_annotation.npy")
pred_refined3=np.load("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/DC_annotation.npy")
pred_refined4=np.load("/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CXC_annotation.npy")
pred_refined_list=[pred_refined1, pred_refined2, pred_refined3, pred_refined4]
#Read in cluster_density information
with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/B_annotation_c_d.pkl', 'rb') as f: c_d1 = pickle.load(f)

with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CD_annotation_c_d.pkl', 'rb') as f: c_d2 = pickle.load(f)

with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/DC_annotation_c_d.pkl', 'rb') as f: c_d3 = pickle.load(f)

with open('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/CXC_annotation_c_d.pkl', 'rb') as f: c_d4 = pickle.load(f)

cluster_density_list=[c_d1, c_d2, c_d3, c_d4]

cnt_color = clr.LinearSegmentedColormap.from_list('red', ["#EAE7CC", '#BA0000'], N=256)
num_required=3
tls_score=ST.TLS_detection(pred_refined_list, cluster_density_list, num_required, cnt_color)
img_tls=ST.plot_TLS_score(img, resize_factor, binary,tls_score, cnt_color)

cv2.imwrite('/home/dingsq/dsq/MGCL-ST-project/Result/CSCC/TLS_score.jpg', img_tls)